# Create the main data for analysis

In [1]:
# packages import
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

In [2]:
print(pd.__version__)
print(np.__version__)
import matplotlib
print(matplotlib.__version__)

1.5.0
1.23.3
3.6.0


## Data import
First, we import the filtered tables

In [3]:
data = pd.read_csv('data/filtered/postcode_filtered.zip')
pets = pd.read_csv('data/filtered/pets_filtered.zip')
flood = pd.read_csv('data/filtered/flood_risk_filtered.zip')
imd = pd.read_csv('data/filtered/imd_filtered.zip')
elevation = pd.read_csv('data/filtered/elevation_filtered.zip')

In [4]:
rows_starting = data.shape[0]

## Join data
### Create district and sector data
From the postcode data, we create district and sector datasets to make sure we have the full list of districts and sectors to join other tables.

In [5]:
# Let's create the district and sector columns
data[['district', 'dis1']] = data['pcds'].str.split(pat = ' ',n = 1, expand=True)
data.drop(['dis1'], axis = 1, inplace = True)

data['sector'] = data['district'].str[0:2]
data['sector'] = data['sector'].str.replace(
    pat = r"[0-9]+", 
    repl = "",
    regex = True
)

In [6]:
# create data with one row per district
district_data = data[['district','sector']].\
    drop_duplicates().\
    reset_index(drop = True).copy()

In [7]:
# create data with one row per sector
lsoa_data = data[['lsoa11']].\
    drop_duplicates().\
    reset_index(drop = True).copy()

### Create full data
Now, let's create full tables, as most of them do not cover all postcodes/districts/sectors.
#### Pets

In [8]:
# join the district data to pets
pets_pcds = district_data.merge(
    pets,
    on = 'district',
    how = 'left'
)

# let's take the districts that do contain values
pets_existing = pets_pcds[~pets_pcds['estimated_cat_population'].isna()].copy()
pets_existing['pets_value_from'] = 'raw_district'

# rows that have missing values
pets_missing = pets_pcds[pets_pcds['estimated_cat_population'].isna()].copy()

# # group the existing values by sector and average the values
pets_sector_avg = pets_existing.groupby(['sector'])\
    .agg({
        'estimated_cat_population': 'mean',
        'dog_per_household_lower95': 'mean'
        }
    )
pets_missing = pets_missing[['district','sector']].merge(
    pets_sector_avg,
    how = 'inner',
    on = 'sector'
)
pets_missing['pets_value_from'] = 'avg_sector'


# put data back together
pets_pcds = pd.concat(
    [pets_existing,pets_missing]
)

# clean memory
del pets_existing, pets_missing, pets_sector_avg

#### IMD

In [9]:
imd.rename(
    columns = {'lsoa':'lsoa11'},
    inplace = True
)

# join the lsoa data to IMD data
lsoa_data = lsoa_data.merge(
    imd,
    on = 'lsoa11',
    how = 'inner'
)

del imd

#### Flood

In [10]:
# merge on all postcodes
flood_pcd = data[['pcds', 'district','sector']].merge(
    flood,
    how = 'left',
    left_on = 'pcds',
    right_on = 'postcode'
)

# CREATE SUB TABLES BY DISTRICT, OR SECTOR

# take the subdata where values are not missing and group by district
flood_dst = flood_pcd[ ~flood_pcd['flood_risk_int'].isna()]\
    .groupby(['district'])\
    .agg({
        'flood_risk_int': 'mean',
        'risk_for_insurance_int': 'mean'
        }
    )
# round to nearest int
flood_dst['flood_risk_int'] = np.round(flood_dst['flood_risk_int'],0)
flood_dst['risk_for_insurance_int'] = np.round(flood_dst['risk_for_insurance_int'],0)

# take the subdata where values are not missing and group by sector
flood_sct = flood_pcd[ ~flood_pcd['flood_risk_int'].isna()]\
    .groupby(['sector'])\
    .agg({
        'flood_risk_int': 'mean',
        'risk_for_insurance_int': 'mean'
        }
    )
# round to nearest int
flood_sct['flood_risk_int'] = np.round(flood_sct['flood_risk_int'],0)
flood_sct['risk_for_insurance_int'] = np.round(flood_sct['risk_for_insurance_int'],0)

In [11]:
### CREATE THE SUBSETS OF OUTPUT TABLE

# by postcode, when value exist by postcode
flood_pcd_e = flood_pcd[ ~flood_pcd['flood_risk_int'].isna()].copy()
flood_pcd_e['flood_value_from'] = 'raw_postcode'

# when value missing by postcode
flood_pcd_m = flood_pcd[ flood_pcd['flood_risk_int'].isna()].copy()

# join flood_pcd_m to district values
flood_pcd_m_dst = flood_pcd_m[["pcds","district", "sector"]].merge(
    flood_dst,
    on = 'district',
    how = 'left'
)

# by district, when value exist averaged by district
flood_pcd_m_dst_e = flood_pcd_m_dst[ ~flood_pcd_m_dst['flood_risk_int'].isna()].copy()
flood_pcd_m_dst_e['flood_value_from'] = 'average_district'

# when value is missing by district
flood_pcd_m_dst_m = flood_pcd_m_dst[ flood_pcd_m_dst['flood_risk_int'].isna()].copy()

# join flood_sct to sector values
flood_pcd_m_dst_m_sct = flood_pcd_m_dst_m[["pcds","sector"]].merge(
    flood_sct,
    on = "sector",
    how = "left"
)

# by sector, when value exist averaged by sector
flood_pcd_m_dst_m_sct_e = flood_pcd_m_dst_m_sct[ ~flood_pcd_m_dst_m_sct['flood_risk_int'].isna()].copy()
flood_pcd_m_dst_m_sct_e['flood_value_from'] = 'average_sector'

# when value is missing by sector
flood_pcd_m_dst_m_sct_m = flood_pcd_m_dst_m_sct[ flood_pcd_m_dst_m_sct['flood_risk_int'].isna()].copy()

print(f"We have {flood_pcd_m_dst_m_sct_m.shape[0]} rows with missing sector values of flood")

# group together
flood_final = pd.concat(
    [
        flood_pcd_e,
        flood_pcd_m_dst_e,
        flood_pcd_m_dst_m_sct_e
    ]
)

# delete un-necessary 
# del (flood_dst, flood_sct, flood_pcd_e, flood_pcd_m, flood_pcd_m_dst,
#      flood_pcd_m_dst_e, flood_pcd_m_dst_m, flood_pcd_m_dst_m_sct,
#      flood_pcd_m_dst_m_sct_e, flood_pcd_m_dst_m_sct_m)

We have 0 rows with missing sector values of flood


#### Elevation

In [12]:
# merge on all postcodes
elev_pcd = data[['pcds', 'district','sector']].merge(
    elevation,
    how = 'left',
    left_on = 'pcds',
    right_on = 'postcode'
)

# CREATE SUB TABLES BY DISTRICT, OR SECTOR

# take the subdata where values are not missing and group by district
elev_dst = elev_pcd[ ~elev_pcd['elevation'].isna()]\
    .groupby(['district'])\
    .agg({
        'elevation': 'mean'
        }
    )
# round to nearest int
elev_dst['elevation'] = np.round(elev_dst['elevation'],0)

# take the subdata where values are not missing and group by sector
elev_sct = elev_pcd[ ~elev_pcd['elevation'].isna()]\
    .groupby(['sector'])\
    .agg({
        'elevation': 'mean'
        }
    )
# round to nearest int
elev_sct['elevation'] = np.round(elev_sct['elevation'],0)


In [13]:
### CREATE THE SUBSETS OF OUTPUT TABLE

# by postcode, when value exist by postcode
elev_pcd_e = elev_pcd[ ~elev_pcd['elevation'].isna()].copy()
elev_pcd_e['elevation_value_from'] = 'raw_postcode'

# when value missing by postcode
elev_pcd_m = elev_pcd[ elev_pcd['elevation'].isna()].copy()

# join elev_pcd_m to district values
elev_pcd_m_dst = elev_pcd_m[["pcds","district", "sector"]].merge(
    elev_dst,
    on = 'district',
    how = 'left'
)

# by district, when value exist averaged by district
elev_pcd_m_dst_e = elev_pcd_m_dst[ ~elev_pcd_m_dst['elevation'].isna()].copy()
elev_pcd_m_dst_e['elevation_value_from'] = 'average_district'

# when value is missing by district
elev_pcd_m_dst_m = elev_pcd_m_dst[ elev_pcd_m_dst['elevation'].isna()].copy()

# join elev_sct to sector values
elev_pcd_m_dst_m_sct = elev_pcd_m_dst_m[["pcds","sector"]].merge(
    elev_sct,
    on = "sector",
    how = "left"
)

# by sector, when value exist averaged by sector
elev_pcd_m_dst_m_sct_e = elev_pcd_m_dst_m_sct[ ~elev_pcd_m_dst_m_sct['elevation'].isna()].copy()
elev_pcd_m_dst_m_sct_e['elevation_value_from'] = 'average_sector'

# when value is missing by sector
elev_pcd_m_dst_m_sct_m = elev_pcd_m_dst_m_sct[ elev_pcd_m_dst_m_sct['elevation'].isna()].copy()

print(f"We have {elev_pcd_m_dst_m_sct_m.shape[0]} rows with missing sector values of elevation")

# group together
elev_final = pd.concat(
    [
        elev_pcd_e,
        elev_pcd_m_dst_e,
        elev_pcd_m_dst_m_sct_e
    ]
)

# delete un-necessary 
del (elev_dst, elev_sct, elev_pcd_e, elev_pcd_m, elev_pcd_m_dst,
     elev_pcd_m_dst_e, elev_pcd_m_dst_m, elev_pcd_m_dst_m_sct,
     elev_pcd_m_dst_m_sct_e, elev_pcd_m_dst_m_sct_m)

We have 0 rows with missing sector values of elevation


#### Full join

In [14]:
data = data\
    .merge(
        pets_pcds[['district','estimated_cat_population','dog_per_household_lower95','pets_value_from']],
        on = 'district',
        how = 'left'
    )\
    .merge(
        lsoa_data,
        on = 'lsoa11',
        how = 'left'
    )\
    .merge(
        flood_final[['pcds','flood_risk_int','risk_for_insurance_int','flood_value_from']],
        on = 'pcds',
        how = 'left'
    )\
    .merge(
        elev_final[['pcds','elevation','elevation_value_from']],
        on = 'pcds',
        how = 'left'
    )

In [15]:
# check number of rows didn't change
data.shape[0] - rows_starting == 0

True

In [16]:
# check data quality
print("Describe:")
data.describe()


Describe:


,lat,long,estimated_cat_population,dog_per_household_lower95,imd_global_rank,imd_global_decile,imd_income_rank,imd_income_decile,imd_employment_rank,imd_employment_decile,...,imd_health_decile,imd_crime_rank,imd_crime_decile,imd_services_rank,imd_services_decile,imd_living_environment_rank,imd_living_environment_decile,flood_risk_int,risk_for_insurance_int,elevation
count,2.191924e+06,2.191924e+06,2.191924e+06,2.191924e+06,2.191924e+06,2.191924e+06,2.191924e+06,2.191924e+06,2.191924e+06,2.191924e+06,...,2.191924e+06,2.191924e+06,2.191924e+06,2.191924e+06,2.191924e+06,2.191924e+06,2.191924e+06,2.191924e+06,2.191924e+06,2.191924e+06
mean,5.232197e+01,-1.218408e+00,4.810207e+03,4.108048e-01,1.631906e+04,5.472304e+00,1.671897e+04,5.587201e+00,1.693034e+04,5.652413e+00,...,5.640168e+00,1.624871e+04,5.448369e+00,1.553529e+04,5.231367e+00,1.485458e+04,5.028880e+00,1.701920e-01,9.312823e-03,6.455306e+01
std,1.153853e+00,1.228830e+00,3.405291e+03,3.085774e-01,9.300788e+03,2.817771e+00,9.426270e+03,2.850853e+00,9.589021e+03,2.893218e+00,...,2.947949e+00,9.735335e+03,2.942602e+00,9.474755e+03,2.866613e+00,9.613661e+03,2.910263e+00,6.036697e-01,9.605258e-02,5.085643e+01
min,4.989198e+01,-6.352647e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+01
25%,5.145023e+01,-2.118069e+00,2.359460e+03,1.955238e-01,8.428000e+03,3.000000e+00,8.628000e+03,3.000000e+00,8.734000e+03,3.000000e+00,...,3.000000e+00,7.566000e+03,3.000000e+00,7.255000e+03,3.000000e+00,6.293000e+03,2.000000e+00,0.000000e+00,0.000000e+00,2.400000e+01
50%,5.203282e+01,-1.254042e+00,4.072930e+03,3.287939e-01,1.638000e+04,5.000000e+00,1.690550e+04,6.000000e+00,1.718100e+04,6.000000e+00,...,6.000000e+00,1.629000e+04,5.000000e+00,1.510300e+04,5.000000e+00,1.415250e+04,5.000000e+00,0.000000e+00,0.000000e+00,5.000000e+01
75%,5.338127e+01,-1.992340e-01,6.498840e+03,5.492530e-01,2.419400e+04,8.000000e+00,2.478800e+04,8.000000e+00,2.525500e+04,8.000000e+00,...,8.000000e+00,2.470500e+04,8.000000e+00,2.358300e+04,8.000000e+00,2.305200e+04,8.000000e+00,0.000000e+00,0.000000e+00,9.000000e+01
max,5.579742e+01,1.760443e+00,2.354445e+04,5.274867e+00,3.284400e+04,1.000000e+01,3.284400e+04,1.000000e+01,3.284400e+04,1.000000e+01,...,1.000000e+01,3.284400e+04,1.000000e+01,3.284400e+04,1.000000e+01,3.284400e+04,1.000000e+01,4.000000e+00,1.000000e+00,5.700000e+02


In [17]:
print("Missing values:")
data.isna().sum()[data.isna().sum()>0]

Missing values:


Series([], dtype: int64)

In [18]:
# Save final data
data.to_csv(
    "data/main_data.zip", 
    index = False,
    compression = 'zip'
)